# Поиск аномалий

Рассматривается вариант обучения без учителя на данных  [superstore](https://community.tableau.com/s/question/0D54T00000CWeX8SAL/sample-superstore-sales-excelxls)

### Загрузка библиотек и данных

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy as sp
import seaborn as sns

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from sklearn.ensemble import IsolationForest

%matplotlib inline

In [ ]:
os.chdir(os.path.join('..', '..'))

In [ ]:
from definitions import DATA_DIR
from src.utils.plots import plot_ad

In [ ]:
SEED = 42

np.random.seed(SEED)

In [ ]:
df = pd.read_excel(os.path.join(DATA_DIR, 'superstore.xls'))

### Рассмотрим одномерные распределения и аномалии

Распределение продаж

In [ ]:
fig = plt.scatter(range(df.shape[0]), np.sort(df['Sales'].values))
plt.xlabel('Индекс')
plt.ylabel('Продажи')
plt.show(fig)

In [ ]:
fig = sns.histplot(df['Sales'])
plt.title('Распределение продаж')
plt.xlabel('Продажи')
fig.set(xlim=(0, 1500))
plt.show(fig)

Явно видно что раcпределение не является нормальным.

Также можно предположить, что регион с низкой вероятностю будет являться аномалией

Используем алгоритм Isolation Forest для поиска аномали в данном распределении, со следующим порядком действий
- Обучение IsolationForest с использованием данных о продажах
- Вычисление значения аномалии для каждого наблюдения  
- Классификация каждого наблюдения как аномалии или нормы
- Визуализация регионов

In [ ]:
isolation_forest = IsolationForest(n_estimators=100, contamination = 0.1)
isolation_forest.fit(df['Sales'].values.reshape(-1, 1))

xx = np.linspace(df['Sales'].min(), df['Sales'].max(), len(df)).reshape(-1,1)
anomaly_score = isolation_forest.decision_function(xx)
outlier = isolation_forest.predict(xx)

fig, ax = plt.subplots(figsize=(10,4))
plt.plot(xx, anomaly_score, label='оценка аномалии')
plt.fill_between(xx.T[0], np.min(anomaly_score), np.max(anomaly_score), 
                 where=outlier==-1, color='r', 
                 alpha=.4, label='аномальная область')
plt.legend()
plt.ylabel('Оценка аномалии')
plt.xlabel('Продажи')
plt.title(f'Аномальная часть для продаж начинается с {outlier.tolist().index(-1)}')
plt.show();

Проверяем значения дохода схожим методом

In [ ]:
fig = plt.scatter(range(df.shape[0]), np.sort(df['Profit'].values))
plt.xlabel('Индекс')
plt.ylabel('Доход')
plt.show(fig)

In [ ]:
fig = sns.histplot(df['Profit'])
plt.title("Распределение доходов")
fig.set(xlim=(-500, 500))
plt.show(fig)

Два аномальных региона -- отрицательный и положительный

In [ ]:
isolation_forest = IsolationForest(n_estimators=100, contamination = 0.1)
isolation_forest.fit(df['Profit'].values.reshape(-1, 1))

xx = np.linspace(df['Profit'].min(), df['Profit'].max(), len(df)).reshape(-1,1)
anomaly_score = isolation_forest.decision_function(xx)
outlier = isolation_forest.predict(xx)

plt.figure(figsize=(10,4))
plt.plot(xx, anomaly_score, label='оценка аномалии')
plt.fill_between(xx.T[0], np.min(anomaly_score), np.max(anomaly_score), 
                 where=outlier==-1, color='r', 
                 alpha=.4, label='аномальная область')
plt.legend()
plt.ylabel('Оценка аномалии')
plt.xlabel('Продажи')
plt.show();

### Многомерный случай

Рассмотрим соотношение доходов и продаж

In [ ]:
fig = sns.regplot(x="Sales", y="Profit", data=df)
plt.ylabel('Доход')
plt.xlabel('Продажи')
plt.show(fig)

Используется библиотека [PyOD](https://pyod.readthedocs.io/en/latest/)

В данном случае аномальным считаем 1% от общего числа данных 

In [ ]:
outliers = 0.01

classifiers = {
    'Cluster-based Local Outlier Factor (CBLOF)':
        CBLOF(contamination=outliers,
              check_estimator=False, random_state=SEED),
    'Histogram-base Outlier Detection (HBOS)': HBOS(
        contamination=outliers),
    'Isolation Forest': IForest(contamination=outliers,
                                behaviour="new",
                                random_state=SEED),
    'K Nearest Neighbors (KNN)': KNN(
        contamination=outliers),
    'Average KNN': KNN(method='mean',
                       contamination=outliers),
    'Local Outlier Factor (LOF)':
        LOF(n_neighbors=35, contamination=outliers),
    'Minimum Covariance Determinant (MCD)': MCD(
        contamination=outliers, random_state=SEED),
    'One-class SVM (OCSVM)': OCSVM(contamination=outliers),
    'Principal Component Analysis (PCA)': PCA(
        contamination=outliers, random_state=SEED)
}

In [ ]:
fig = plot_ad(df = df, models = classifiers, outliers_fraction = outliers)
fig